In [3]:
import sys, os
sys.path.append(os.path.abspath("../"))  
import pandas as pd
from core.factors import *

In [4]:
df = pd.read_parquet("../data/twse_2020_to_2025_sector.parquet")

In [5]:
components = [
    "1303","2002","2303","2330","2881","1301","2324","2409","2880","2882","2886","2887",
    "2891","1216","1402","1605","2323","2352","2371","2408","2603","2609","2610","2801",
    "2890","1101","1326","2317","2337","2357","2382","2412","2884","2885","2892","3481",
    "2353","2454","2915","3231","1102","1210","1312","1314","1319","1440","1504","1590",
    "1718","1722","2006","2014","2027","2049","2059","2105","2201","2301","2308","2312",
    "2313","2331","2332","2340","2344","2347","2354","2376","2377","2379","2385","2392",
    "2393","2401","2404","2449","2455","2457","2458","2474","2481","2485","2489","2492",
    "2498","2515","2520","2542","2548","2605","2618","2834","2913","3006","3008","3019",
    "3034","3035","3036","3037","3042","3189","3376","3380","3443","3533","3653","3673",
    "3702","4938","5534","6005","6153","6176","6213","6239","6271","6278","6282","6285",
    "8039","8163","9904","9939","9945","1477","1802","2328","3044","3045","3406","6269",
    "9914","5880","2356","2883","4904","4958","5871","1476","2327","8046","1707","2355",
    "2360","2439","6257","9938","1565","3105","3152","3260","3264","3691","4123","5009",
    "5347","5371","5483","6121","6147","8044","8069","8299","0050","006205","006206","2231",
    "6116","6279","00636","00639","00643","2345","6414","1536","1909","3081","3552","6274",
    "6488","6510","3711","4162","4736","5425","0056","2633","5269","3529","2383","6173",
    "6182","8436","5457","8358","8086","3706","3324","6669","3293","5274","3714","2606",
    "3532","1907","3374","1717","1904","3078","2441","8150","2338","2388","2615","6547",
    "6770","3017","5388","2634","4128","4919","00878","1609","2368","6443","4743","6245",
    "5904","8454","00923","00679B","1795","1905","1513","00893","00719B","3005","8112",
    "00919","00929","00772B","00940","9958","3680","1503","6139","6188","5876","6505",
    "6526","00937B","00687B","3661","6472","00757","2486","6757","6223","2329","6290",
    "1608","2367","2421"
]

df = df[df['stock_id'].isin(components)]

In [6]:
df.stock_id.unique()

array(['1101', '1102', '1210', '1216', '1301', '1303', '1312', '1314',
       '1319', '1326', '1402', '1440', '1476', '1477', '1503', '1504',
       '1513', '1536', '1590', '1605', '1608', '1609', '1707', '1717',
       '1718', '1722', '1795', '1802', '1904', '1905', '1907', '1909',
       '2002', '2006', '2014', '2027', '2049', '2059', '2105', '2201',
       '2231', '2301', '2303', '2308', '2312', '2313', '2317', '2323',
       '2324', '2327', '2328', '2329', '2330', '2331', '2332', '2337',
       '2338', '2340', '2344', '2345', '2347', '2352', '2353', '2354',
       '2355', '2356', '2357', '2360', '2367', '2368', '2371', '2376',
       '2377', '2379', '2382', '2383', '2385', '2388', '2392', '2393',
       '2401', '2404', '2408', '2409', '2412', '2421', '2439', '2441',
       '2449', '2454', '2455', '2457', '2458', '2474', '2481', '2485',
       '2486', '2489', '2492', '2498', '2515', '2520', '2542', '2548',
       '2603', '2605', '2606', '2609', '2610', '2615', '2618', '2633',
      

In [90]:
high = df.pivot(index="date", columns="stock_id", values="high").sort_index().shift(1)
low  = df.pivot(index="date", columns="stock_id", values="low").sort_index().shift(1)
vol = df.pivot(index="date", columns="stock_id", values="volume").sort_index().shift(1)
close = df.pivot(index="date", columns="stock_id", values="close").sort_index().shift(1)
open = df.pivot(index="date", columns="stock_id", values="open").sort_index().shift(1)

In [91]:
def apply_liquidity_filter(factor: pd.DataFrame,
                           vol: pd.DataFrame,
                           close: pd.DataFrame,
                           top_n: int = 100) -> pd.DataFrame:
    """Keep only top-N stocks by (volume × close) each day."""
    dollar_vol = vol * close
    ranks = dollar_vol.rank(axis=1, ascending=False, method="first")
    mask = ranks <= top_n
    return factor.where(mask)


In [92]:
from core.factors import *

# close: DataFrame (date×asset)
delta1 = ts_delta(close, 1)
tsmin4 = ts_min(delta1, 4)
tsmax4 = ts_max(delta1, 4)

# condition logic
cond1 = greater(tsmin4, 0)
cond2 = less(tsmax4, 0)

# nested ternary condition
temp = where(cond1, delta1, where(cond2, delta1, mul(-1, delta1)))

# final cross-sectional rank
alpha10 = rank_cs(temp)

In [93]:
rank_high = rank_cs(high)
rank_vol  = rank_cs(vol)

corr_3d = corr_ts(rank_high, rank_vol, d=3)

rank_corr = rank_cs(corr_3d)

sum3 = ts_sum(rank_corr, d=3)

alpha15 = -1 * sum3

In [94]:
amount = vol*close                        

d10 = ts_delta(close, 10)
r1  = rank_cs(d10)
r2  = rank_cs(r1)
inner = -1 * r2

# decay_linear(..., 10) then triple rank
dec  = decay_linear(inner, d=10)
A = rank_cs(rank_cs(rank_cs(dec)))

# ---------- part B ----------
# rank((-1) * delta(close, 3))
d3 = ts_delta(close, 3)
B  = rank_cs(-1 * d3)

# ---------- part C ----------
# sign(scale(correlation(adv20, low, 12)))
adv20 = adv(amount, d=20)
corr12 = corr_ts(adv20, low, d=12)
C = np.sign(scale_cs(corr12))  # sign after cross-sectional scale

# ---------- alpha31 ----------
alpha31 = A + B + C

In [95]:
intraday_ret = (close - open) / open.replace(0, np.nan)

alpha2 = -1 * corr_ts(
    rank_cs(ts_delta(log(vol), 2)),   # rank(delta(log(volume), 2))
    rank_cs(intraday_ret),               # rank((close-open)/open)
    d=6                                  # correlation over 6 days
)

In [96]:
alpha40 = (-1*rank_cs(ts_std(high,10)))*corr_ts(high,vol,10)

In [97]:
vwap    = amount / vol                          

alpha5 = rank_cs(open - ts_sum(vwap, 10) / 10) * (-1 * abs_(rank_cs(close - vwap)))


In [98]:
factor_merge = normalize(alpha40)+normalize(alpha15)+normalize(alpha31)+normalize(alpha10)+normalize(alpha2)

In [99]:
import pandas as pd
from core.backtest import backtest_core
# your factor (date×asset), expected returns matrix (date×asset)
# e.g. expected return = next-day close-to-close return
ret = close.pct_change().shift(-1)          # target return
factor =normalize(alpha31)+normalize(alpha10)

/tmp/ipykernel_56999/3168892770.py:5: FutureWarning:

The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.



In [100]:
ret

stock_id,1101,1102,1210,1216,1301,1303,1312,1314,1319,1326,...,8112,8150,8163,8454,9904,9914,9938,9939,9945,9958
date,,,,,,,,,,,,,,,,,,,,,
2020-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-03,-0.003401,-0.002045,-0.017361,0.008086,0.019076,0.008264,0.013055,-0.004107,-0.004283,0.010262,...,0.004934,-0.010130,0.005675,0.000000,0.000000,0.005747,0.002364,0.011236,-0.001107,-0.016529
2020-01-06,-0.011377,-0.010246,-0.010601,-0.012032,0.004926,-0.002732,-0.018041,-0.035052,-0.021505,-0.001129,...,0.006547,-0.023392,-0.006772,0.000000,-0.010076,0.000000,0.023585,-0.012698,-0.011086,-0.004202
2020-01-07,0.003452,0.001035,0.023810,-0.006766,-0.004902,-0.005479,-0.015748,-0.012821,0.017582,-0.012429,...,0.004878,-0.020958,-0.019318,0.003448,-0.002545,0.002857,-0.012673,-0.004823,0.000000,-0.011252
2020-01-08,-0.004587,0.004137,-0.005814,-0.010899,-0.014778,-0.013774,-0.016000,-0.007576,-0.017279,-0.009153,...,0.008091,-0.003058,-0.010429,-0.001718,-0.012755,-0.002849,-0.009335,0.000000,-0.011211,-0.032717
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-09-01,-0.004454,-0.018893,0.000000,0.000000,-0.003851,-0.016204,-0.001018,0.004196,0.014778,0.006645,...,-0.023148,-0.014374,-0.024862,-0.017787,-0.019064,-0.008811,-0.011173,-0.018051,-0.009950,-0.029973
2025-09-02,-0.011186,-0.001376,0.005650,0.010336,-0.019330,-0.061176,-0.011213,-0.006964,-0.019417,-0.036304,...,-0.007109,-0.010417,0.001416,0.020121,-0.007067,0.013333,0.001883,-0.014706,-0.001675,-0.011236
2025-09-03,0.011312,0.002755,-0.003745,-0.003836,-0.007884,-0.015038,-0.015464,-0.004208,-0.009901,-0.017123,...,0.007160,0.098947,0.004243,-0.003945,0.005338,-0.030702,0.007519,-0.007463,0.010067,-0.017045


In [101]:
import pandas as pd

df_sector = pd.read_parquet("../data/twse_2020_to_2025_sector.parquet")
sector_map = df_sector.drop_duplicates(subset=["stock_id"])[["stock_id", "sector"]]
sector_map = sector_map.set_index("stock_id")["sector"]


In [123]:
factor_liquid = apply_liquidity_filter(alpha31, vol, close, top_n=100)


In [124]:
factor =normalize(alpha31)+normalize(alpha10)

In [135]:

# optional benchmark (Series): e.g. equally-weighted market or 0050 ETF daily return
bm = ret.mean(axis=1)                       # simple EW market as benchmark


report = backtest_core(
    factor=alpha31,
    exp_ret=ret,
    period_of_year=252,
    benchmark=bm,
    fee=0.0000,
    exec_lag=0,                 # use next-bar execution if you want
    sector_map=sector_map,
    neutralize=True,           # turn on if you pass sector_map
    preprocess='none', # or "rank" / "none"
    leverage=1.0,
    cap=0.05,                   # per-name cap (optional)
    return_weights=True,
    plot=True,
)

print(report.summary)
turnover = report.weights.pipe(lambda w: (w.diff().abs().sum(axis=1)).mean())
print("Avg turnover:", round(float(turnover), 6))


/home/echiu/Desktop/vscode/long_short_portfolio2/core/backtest.py:51: FutureWarning:

DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.



                      Performance Benchmark
Cumprod Total Returns     37.97 %  104.64 %
Cumsum Total Returns      32.47 %   82.85 %
Sharpe Ratio                 1.84      0.75
Sortino Ratio                3.00      0.83
Annualized Ret             6.05 %   13.98 %
Max Drawdown               3.09 %   33.40 %
Volatility                 3.22 %   20.16 %
STD                        0.20 %    1.27 %
Avg turnover: 0.480752


In [73]:
ret

stock_id,1101,1102,1210,1216,1301,1303,1312,1314,1319,1326,...,8112,8150,8163,8454,9904,9914,9938,9939,9945,9958
date,,,,,,,,,,,,,,,,,,,,,
2020-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-03,-0.003401,-0.002045,-0.017361,0.008086,0.019076,0.008264,0.013055,-0.004107,-0.004283,0.010262,...,0.004934,-0.010130,0.005675,0.000000,0.000000,0.005747,0.002364,0.011236,-0.001107,-0.016529
2020-01-06,-0.011377,-0.010246,-0.010601,-0.012032,0.004926,-0.002732,-0.018041,-0.035052,-0.021505,-0.001129,...,0.006547,-0.023392,-0.006772,0.000000,-0.010076,0.000000,0.023585,-0.012698,-0.011086,-0.004202
2020-01-07,0.003452,0.001035,0.023810,-0.006766,-0.004902,-0.005479,-0.015748,-0.012821,0.017582,-0.012429,...,0.004878,-0.020958,-0.019318,0.003448,-0.002545,0.002857,-0.012673,-0.004823,0.000000,-0.011252
2020-01-08,-0.004587,0.004137,-0.005814,-0.010899,-0.014778,-0.013774,-0.016000,-0.007576,-0.017279,-0.009153,...,0.008091,-0.003058,-0.010429,-0.001718,-0.012755,-0.002849,-0.009335,0.000000,-0.011211,-0.032717
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-09-01,-0.004454,-0.018893,0.000000,0.000000,-0.003851,-0.016204,-0.001018,0.004196,0.014778,0.006645,...,-0.023148,-0.014374,-0.024862,-0.017787,-0.019064,-0.008811,-0.011173,-0.018051,-0.009950,-0.029973
2025-09-02,-0.011186,-0.001376,0.005650,0.010336,-0.019330,-0.061176,-0.011213,-0.006964,-0.019417,-0.036304,...,-0.007109,-0.010417,0.001416,0.020121,-0.007067,0.013333,0.001883,-0.014706,-0.001675,-0.011236
2025-09-03,0.011312,0.002755,-0.003745,-0.003836,-0.007884,-0.015038,-0.015464,-0.004208,-0.009901,-0.017123,...,0.007160,0.098947,0.004243,-0.003945,0.005338,-0.030702,0.007519,-0.007463,0.010067,-0.017045


In [89]:
factor

stock_id,1101,1102,1210,1216,1301,1303,1312,1314,1319,1326,...,8112,8150,8163,8454,9904,9914,9938,9939,9945,9958
date,,,,,,,,,,,,,,,,,,,,,
2020-01-02,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2020-01-03,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2020-01-06,0.205429,0.125043,0.911031,-1.312957,-1.545180,-1.348683,-1.071801,-0.116112,0.348335,-1.420137,...,-0.875304,0.491242,-1.071801,-0.276882,-0.276882,-1.473727,-1.000348,-1.384410,-0.089317,1.134323
2020-01-07,0.303638,0.303638,0.241124,0.651928,-1.527118,-0.509039,-0.017861,-0.044653,0.750163,-0.973426,...,-1.437813,0.634067,-0.241124,-1.303855,0.178610,-1.303855,-1.661076,0.607275,0.303638,-0.062514
2020-01-08,-1.125478,-0.991492,-1.518502,0.759251,0.759251,0.678860,0.491280,0.026797,-1.455975,1.036154,...,-1.125478,0.848575,0.920033,-1.518502,-0.125053,-1.384516,1.036154,0.535942,-0.652063,0.884304
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-09-01,-0.783740,-1.221727,1.729481,1.460768,1.383684,-1.982395,0.527935,0.679285,1.624189,0.162322,...,1.762813,-0.817186,-0.064508,3.302146,0.161590,-2.760731,-1.213382,-1.559001,-2.145310,2.717874
2025-09-02,-0.940184,-2.335287,0.371739,-0.038445,-0.150960,-1.343639,-0.112797,-0.233718,1.091963,-0.782409,...,1.077792,0.804068,0.749576,2.792914,0.639567,-1.447710,-1.747645,0.118565,-1.720730,0.373483
2025-09-03,-0.679075,-1.148270,0.134313,-0.545081,-0.199228,0.182554,0.340281,0.767076,-0.224994,1.538937,...,-0.612669,1.345721,0.256783,0.061548,1.042956,-3.230682,-1.902050,0.621129,0.892522,-1.941841


In [20]:
def apply_liquidity_filter(factor: pd.DataFrame,
                           vol: pd.DataFrame,
                           close: pd.DataFrame,
                           top_n: int = 100) -> pd.DataFrame:
    """Keep only top-N stocks by (volume × close) each day."""
    dollar_vol = vol * close
    ranks = dollar_vol.rank(axis=1, ascending=False, method="first")
    mask = ranks <= top_n
    return factor.where(mask)


In [18]:
factor_liquid = apply_liquidity_filter(factor_merge, vol, close, top_n=500)


In [19]:
factor_liquid

stock_id,1101,1102,1103,1104,1108,1109,1110,1201,1203,1210,...,9939,9940,9941,9942,9943,9944,9945,9946,9955,9958
date,,,,,,,,,,,,,,,,,,,,,
2020-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-03,0.000000,0.000000,0.00000,0.000000,NaN,0.000000,NaN,0.000000,NaN,0.000000,...,0.000000,NaN,0.000000,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000
2020-01-06,0.236722,0.116452,1.08434,0.421900,NaN,-1.233246,NaN,-0.286357,NaN,1.156884,...,-1.563511,NaN,-1.511967,NaN,NaN,0.700621,-0.286357,0.316902,-1.616965,1.330607
2020-01-07,0.810004,0.810004,-0.19868,NaN,NaN,-0.500521,NaN,0.240709,NaN,0.741230,...,1.127128,NaN,0.810004,0.810004,0.110802,NaN,0.810004,0.240709,-1.669677,0.456582
2020-01-08,-1.202042,-1.047739,-1.38111,-0.043815,NaN,NaN,NaN,0.746752,NaN,-1.573513,...,0.883911,NaN,-1.628758,NaN,1.135368,NaN,-0.516250,-1.506839,1.221092,1.215377
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-09-01,-0.162237,-1.937614,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.796695,...,0.251062,NaN,2.851566,NaN,NaN,NaN,-3.918468,NaN,0.814505,3.526696
2025-09-02,-0.511317,-3.037876,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.051715,...,2.227277,NaN,5.272734,NaN,NaN,NaN,-1.595321,NaN,-2.408792,0.920796
2025-09-03,-0.846071,-4.058582,NaN,0.808187,NaN,NaN,NaN,NaN,NaN,-0.423204,...,3.916166,NaN,4.476954,NaN,NaN,NaN,0.706588,NaN,-4.194378,-3.149871
